In [1]:
from icecream import ic, install
install()
ic.disable()

In [2]:
from spacy import displacy
from spacy import blank
from spacy import load

from MyUtilty.tag2pos import tag2posFactory
from MyUtilty.psoPipe import PendingAspectsOpinionsPipe
from MyUtilty.cost_seg import CostSegmenter
from MyUtilty.emoBankSearch import EmoBankSearch

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-10-31 03:31:39.318252: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-31 03:31:39.342393: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-31 03:31:39.349884: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-31 03:31:39.366057: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to 

In [3]:
from spacy.tokens import Doc
Doc.set_extension("pending_aspects_opinions_i", default=[])
Doc.set_extension("pending_aspects_opinions_text", default=[])
Doc.set_extension("aspect_sentiment_triplets", default=[])

In [4]:
text = [
    "店員服務很好。",
    "畢比登推薦，天下三絕名不虛傳！！ 牛肉麵、排骨飯、小菜、青菜都美味可口。乾淨衛生、有溫度的親切服務、親子友善、環境優美、氣氛佳。家庭或朋友聚餐首選，大推！！",
    # "B4🤭"
]
# text = ["裝潢乾淨有質感，蔬菜新鮮口味不錯。湯飯加15元可以吃到飽，素食店的價位，均消費85起。週一有九折優惠。"]

In [5]:
# nlp = Language(vocab=vocab)
nlp = blank("xx")
nlp.tokenizer = CostSegmenter(nlp.vocab)
nlp.add_pipe("tag2pos")
nlp.add_pipe("pending_aspects_opinions_pipe")

nlp_latin = load("./vec")
EBS = EmoBankSearch(nlp_latin)

100%  47.2 MiB 405.8 KiB/s ETA:  0 s [=========================================]
Decompressing /home/appuser/.hanlp/dep/ctb9_dep_electra_small_20220216_100306.zip to /home/appuser/.hanlp/dep
100%  26.7 KiB  26.7 KiB/s ETA:  0 s [=========================================]
Decompressing /home/appuser/.hanlp/utils/char_table_20210602_202632.json.zip to /home/appuser/.hanlp/utils
100%  41.2 KiB  41.2 KiB/s ETA:  0 s [=========================================]
Decompressing /home/appuser/.hanlp/transformers/electra_zh_small_20210706_125427.zip to /home/appuser/.hanlp/transformers


In [6]:
# nlp.analyze_pipes()

In [7]:
docList = list(nlp.pipe(text))
docList_ebs = list(map(EBS, docList))

In [8]:
displacy.render(docList[1])

In [9]:
for doc in docList_ebs:
    print(f"---doc.text: {doc.text}---")
    # print(
    #     f"---{doc._.pending_aspects_opinions_text}, {doc._.pending_aspects_opinions_i}---"
    # )
    print(f"---{doc._.aspect_sentiment_triplets}---")
    for token in doc:
        print(token, token.pos_, token.tag_, token.dep_, token.head)

---doc.text: 店員服務很好。---
---[('服務', '好', 6.8)]---
店員 NOUN Na nn 服務
服務 VERB VC nsubj 好
很 ADV Dfa advmod 好
好 VERB VH root 好
。 PUNCT PERIODCATEGORY punct 好
---doc.text: 畢比登推薦，天下三絕名不虛傳！！ 牛肉麵、排骨飯、小菜、青菜都美味可口。乾淨衛生、有溫度的親切服務、親子友善、環境優美、氣氛佳。家庭或朋友聚餐首選，大推！！---
---[('畢比登', '推薦', 6.2), ('絕', '名不虛傳', 7.5), ('青菜', '美味', 6.8), ('親子', '友善', 6.6), ('環境', '優美', 6.8), ('氣氛', '佳', 6.4), ('朋友', '聚餐', 6.44), ('牛肉麵', '美味', 6.8), ('排骨飯', '美味', 6.8), ('小菜', '美味', 6.8), ('家庭', '聚餐', 6.44), ('青菜', '可口', 6.4)]---
畢比登 PROPN Nb nsubj 推薦
推薦 VERB VC root 推薦
， PUNCT COMMACATEGORY punct 推薦
天下 NOUN Nc nn 絕
三 NUM Neu nummod 絕
絕 VERB VH nsubj 名不虛傳
名不虛傳 VERB VH ccomp 推薦
！ PUNCT EXCLAMATIONCATEGORY punct 名不虛傳
！ PUNCT EXCLAMATIONCATEGORY punct 名不虛傳
  PUNCT WHITESPACE punct 青菜
牛肉麵 NOUN Na conj 青菜
、 PUNCT PAUSECATEGORY punct 青菜
排骨飯 NOUN Na conj 青菜
、 PUNCT PAUSECATEGORY punct 青菜
小菜 NOUN Na conj 青菜
、 PUNCT PAUSECATEGORY punct 青菜
青菜 NOUN Na nsubj 美味
都 ADV D advmod 美味
美味 VERB VH conj 名不虛傳
可口 VERB VH comod 美味
。 PUNCT PERIODCATEGORY pun

---

In [ ]:
import pandas as pd
from MyUtilty.eveal import ans2aop, all_score
from MyUtilty.pat_discover_engine import PatternDiscoverEngine

In [ ]:
test_gold300 = pd.read_json("/workspaces/ABSA/test_gold300.json", lines=True)
test_gold = test_gold300[test_gold300["answer"].str.len() > 0]
test_gold["context"] = list(map(lambda x: x.split("context: ")[-1].replace("⋯", "。"), test_gold.question.values))
ture_set = list(map(ans2aop, test_gold.answer.values))

In [ ]:
train = pd.read_json("/workspaces/ABSA/train.json", lines=True)
c_train = train[train["answer"].str.len() > 0]
c_train["context"] = list(map(lambda x: x.split("context: ")[-1].replace("⋯", "。"), c_train.question.values))
train_set = list(map(ans2aop, c_train.answer.values))

In [ ]:
test_gold["context"].shape

In [ ]:
docs = list(nlp.pipe(test_gold["context"].values))
docListEBS = list(map(EBS, docs))
# docListSi = list(map(si, docList))

In [ ]:
ture_set[0]

In [ ]:
docListEBS[0]._.pending_aspects_opinions_text

## CTB9_DEP_ELECTRA_SMALL 

[1~5]

(0.3903985178419504,
 0.4438067643043281,
 **0.5693875870506305,**
 0.47232278939710676)

(0.3416190024775145,
 0.38805714837386507,
 **0.5039559675429242,**
 0.4164505191418483)

[1~6]

(0.3969485422909424,
 0.4424210875380512,
 0.5829314103770624,
 0.4772676213666051)

(0.3477103327699727,
 0.38686595157348763,
 0.5169821925256708,
 0.42118658242431717)

In [ ]:
all_score([i._.pending_aspects_opinions_text for i in docListEBS], list(zip(*ture_set))[0])

In [ ]:
all_score([i._.aspect_sentiment_triplets for i in docListEBS], list(zip(*ture_set))[1])

In [ ]:
docListEBS[0].pos_

In [ ]:
for x, doc in enumerate(zip(docListEBS, ture_set)):
    print(f"---idx[{x}]: {doc[0].text} ---")
    # print(f"---idx[{i}]---")
    # print("POS:", doc[0])
    print("Pred:", doc[0]._.aspect_sentiment_triplets)
    print("Gold:", doc[1][1])

In [ ]:
# t_set = set(list(zip(*ture_set))[0][0])
# p_set = set(docListEBS[0]._.pending_aspects_opinions_text)

In [ ]:
# t_set
# t_set - p_set

In [ ]:
pde = PatternDiscoverEngine()

In [ ]:
# ture_set[78][0]

In [ ]:
# pde.pat_serach(docListEBS[78], ture_set[78][0])

In [ ]:
import pickle

In [ ]:
with open("./docListEBS.pkl", 'wb') as f:
    pickle.dump(docListEBS, f)

In [ ]:
with open("./docListEBS.pkl", "rb") as f:
    docListEBS = pickle.load(f)


In [ ]:
docListEBS

In [ ]:
train_set

In [ ]:
err = []
for i, x in enumerate(zip(docListEBS, train_set)):
    # print(f"---idx[{i}]: {x[0].text} ---")
    try:
        pde.pat_serach(x[0], x[1][0])
    except:
        err.append(i)

In [ ]:
docListEBS[0]._.aspect_sentiment_triplets

In [ ]:
pat_set = pde.get_patterns()

In [ ]:
pat_set.most_common(10)

In [ ]:
pat_set

In [ ]:
ture_set[0][0]

In [ ]:
docListEBS[0][0].head

In [ ]:
# displacy.render(docListEBS[0])

In [ ]:
# displacy.render(docListEBS[267])

In [ ]:
# displacy.render(docListEBS[4])

In [ ]:
# displacy.render(docListEBS[15])

In [ ]:
docListEBS[14]._.aspect_sentiment_triplets

In [ ]:
# displacy.render(docListEBS[9])